# Documentação Técnica do Notebook: Extração e Processamento de Dados BI-RADS - Pardini

## Objetivo Principal
**Este notebook realiza a extração, transformação e persistência dos dados de laudos de mamografia do Pardini, com foco na classificação BI-RADS.** O objetivo é identificar e classificar exames de mama segundo o BI-RADS, além de gerar uma lista de ativação para notificação de pacientes com exames em atraso.

## Tecnologias Utilizadas
- **PySpark**: Manipulação de dados, consultas SQL, transformações e persistência em Delta Lake.
- **Octoops**: Monitoramento, alertas e integração com sistemas de controle de execução.
- **Delta Lake**: Persistência dos dados processados.
- **Python (logging, traceback, etc.)**: Controle de fluxo, tratamento de erros e registro de eventos.

## Fluxo de Trabalho/Etapas Principais
1. Instalação de dependências (`octoops`).
2. Configuração do ambiente (reinicialização do kernel e importação de bibliotecas).
3. Definição de tabelas e filtros SQL para seleção incremental e ativação de pacientes.
4. Consulta SQL principal para extração e enriquecimento dos dados dos laudos.
5. Transformações nos DataFrames para cálculo de datas previstas de retorno, classificação e cálculo de diferenças de datas.
6. Persistência dos dados nas tabelas Delta, com tratamento de erros e envio de alertas.
7. Remoção de duplicados na lista de ativação.
8. Impressão da quantidade de registros salvos.

## Dados Envolvidos
- **Fonte**: Tabela `refined.saude_preventiva.pardini_laudos`.
- **Tabelas de destino**: `refined.saude_preventiva.pardini_laudos_mama_birads` e `refined.saude_preventiva.pardini_laudos_mama_birads_ativacao`.
- **Colunas importantes**: `laudo_tratado`, `BIRADS`, `MIN_BIRADS`, `MAX_BIRADS`, `dth_pedido`, `dth_previsao_retorno`, `dias_ate_retorno`, entre outras.

## Resultados/Saídas Esperadas
- DataFrame completo com laudos processados e classificados segundo BI-RADS.
- DataFrame filtrado para ativação de pacientes elegíveis para notificação.
- Persistência dos dados nas tabelas Delta.
- Alertas automáticos em caso de falhas ou ausência de dados.

## Pré-requisitos
- Ambiente Databricks ou Spark configurado.
- Pacotes: `octoops`, permissões de escrita nas tabelas Delta.
- Acesso às tabelas Delta e permissões de escrita.

## Considerações Importantes
- O notebook utiliza expressões regulares avançadas para correta identificação dos valores BI-RADS.
- O merge/upsert garante atualização incremental dos dados sem duplicidade.
- O monitoramento via Octoops/Sentinel facilita rastreabilidade e resposta rápida a falhas.
- O filtro de idade e sexo garante que apenas pacientes do público-alvo sejam analisados.

---
A seguir, cada célula de código é precedida por uma explicação técnica detalhada sobre sua função e impacto no fluxo do notebook.

## Instalação do pacote octoops
Esta célula instala o pacote `octoops`, utilizado para monitoramento, alertas e integração com sistemas de controle de execução. O comando `%pip install` garante que o pacote esteja disponível no ambiente do notebook. Caso o ambiente já possua o pacote, a instalação será ignorada.

In [ ]:
%pip install octoops

## Reinicialização do ambiente Python (Databricks)
Esta célula reinicia o kernel Python no ambiente Databricks, garantindo que todas as dependências recém-instaladas estejam disponíveis. O comando `dbutils.library.restartPython()` é específico do Databricks e não deve ser executado em ambientes fora dele.

In [ ]:
dbutils.library.restartPython()

## Importação de bibliotecas
Importa todas as bibliotecas necessárias para o processamento dos dados:
- **PySpark**: Manipulação de DataFrames, funções SQL, tipos de dados e cálculos de datas.
- **Octoops**: Classes para monitoramento (`OctoOps`, `Sentinel`).
- **Logging, traceback, sys**: Controle de fluxo, registro de eventos e tratamento de erros.
Essas bibliotecas são essenciais para todas as etapas do notebook, desde a extração até a persistência e monitoramento dos dados.

In [ ]:
from pyspark.sql.functions import col, year, month, dayofmonth, when, lit, expr, to_timestamp
from pyspark.sql.types import DateType
from pyspark.sql import DataFrame
from pyspark.sql.functions import datediff, to_date
from datetime import datetime
import logging
import sys
import traceback
from octoops import OctoOps
from octoops import Sentinel

## Configuração do logger
Cria um logger para registrar informações, avisos e erros durante a execução do notebook. O logger é utilizado nas funções de transformação e persistência para facilitar o monitoramento e depuração.

In [ ]:
logger = logging.getLogger(__name__)

## Definição de tabelas e filtros
Define os nomes das tabelas Delta de destino e os filtros SQL utilizados para seleção incremental dos dados e ativação de pacientes. O filtro incremental garante que apenas novos dados sejam processados, enquanto o filtro de ativação seleciona pacientes elegíveis para notificação.

In [ ]:
table_birads = "refined.saude_preventiva.pardini_laudos_mama_birads"
table_birads_ativacao = "refined.saude_preventiva.pardini_laudos_mama_birads_ativacao"
where_clause = ""
 
if spark.catalog.tableExists(table_birads):
    where_clause = f"""
    WHERE        
        flr._datestamp > (
            SELECT MAX(brd._datestamp)
            FROM {table_birads} brd
        )
        
    """
 
filtro_ativacao = """
    WHERE
        eleg.ficha IS NULL
        AND brd.BIRADS IN (1, 2, 3)
        AND flr.sigla_exame IN ('MAMODI','MAMO')
        AND UPPER(flr.sexo_cliente) = 'F'
        AND (
            idade_cliente >= 40 AND idade_cliente < 76
        )
"""

## Consulta SQL principal
Esta célula define e executa a consulta SQL que extrai, processa e enriquece os dados dos laudos de mamografia. O SQL realiza:
- Limpeza e padronização do texto dos laudos;
- Extração dos valores BI-RADS (inclusive algarismos romanos);
- Cálculo de idade do paciente;
- Enriquecimento com dados de retorno elegível;
- Aplicação dos filtros incremental e de ativação.
Os resultados são carregados em dois DataFrames: um completo e outro filtrado para ativação.

In [ ]:
query = """
WITH base AS (
    SELECT
        flr.ficha,
        flr.sigla_exame, 
        flr.id_marca,
        flr.sequencial,
        flr.laudo_tratado,
        REGEXP_EXTRACT_ALL(
            REGEXP_EXTRACT(
                REGEXP_REPLACE(UPPER(flr.laudo_tratado), r'[-:®]|\xa0', ''),
                r'(?mi)(AVALIA[CÇ][AÃ]O|CONCLUS[AÃ]O|IMPRESS[AÃ]O|OPINI[AÃ]O)?(.*)', 2
            ),
            r"(?mi)(BIRADS|CATEGORI[AO]|CATEGORA|CATEGORIA R|CAT\W)\s*(\d+\w*|VI|V|IV|III|II|I)\W*\w?(BIRADS|CATEGORI[AO]|CATEGORA|CATEGORIA R)?(\W|$)", 2
        )        
        AS RAW_BIRADS,
        FILTER(
            TRANSFORM(RAW_BIRADS, x ->
                CASE
                    WHEN x = "I" THEN 1
                    WHEN x = "II" THEN 2
                    WHEN x = "III" THEN 3
                    WHEN x = "IV" THEN 4
                    WHEN x = "V" THEN 5
                    WHEN x = "VI" THEN 6
                    WHEN TRY_CAST(x AS INT) > 6 THEN NULL
                    ELSE REGEXP_REPLACE(x, r'[^0-9]', '')
                END
            ), x -> x IS NOT NULL
        ) AS CAT_BIRADS
    FROM refined.saude_preventiva.pardini_laudos flr
    WHERE
        flr.linha_cuidado = 'mama'
        AND UPPER(flr.sexo_cliente) = 'F' 
        AND flr.sigla_exame IN ('MAMO','MAMODI')
),
 
dados_birads AS (
    SELECT
        *,
        ARRAY_MIN(CAT_BIRADS) AS MIN_BIRADS,
        ARRAY_MAX(CAT_BIRADS) AS MAX_BIRADS,
        TRY_ELEMENT_AT(CAST(CAT_BIRADS AS ARRAY<INT>), -1) AS BIRADS
    FROM base
),
 
dados_laudos AS (
    SELECT
        flr.linha_cuidado,
        flr.id_unidade,
        flr.id_ficha,
        flr.sigla_exame, 
        flr.id_marca,
        flr.sequencial,
        flr.ficha,
        flr.id_cliente,
        flr.pefi_cliente,
        flr.sigla_exame,
        flr.marca,
        flr.laudo_tratado,
        (
          TIMESTAMPDIFF(DAY, flr.dth_nascimento_cliente, CURDATE()) / 365.25
        ) AS idade_cliente,
        flr.sexo_cliente,
        flr.dth_pedido,
        flr._datestamp
    FROM refined.saude_preventiva.pardini_laudos flr
    {where_clause}
)
 
SELECT
    flr.* except(idade_cliente),
    brd.MIN_BIRADS,
    brd.MAX_BIRADS,
    brd.BIRADS,
 
    eleg.dth_pedido        AS dth_pedido_retorno_elegivel,
    eleg.ficha             AS ficha_retorno_elegivel,
    eleg.siglas_ficha      AS siglas_ficha_retorno_elegivel,
    eleg.marca             AS marca_retorno_elegivel,
    eleg.unidade           AS unidade_retorno_elegivel,
    eleg.convenio          AS convenio_retorno_elegivel,
    eleg.valores_exame     AS valores_exame_retorno_elegivel,
    eleg.valores_ficha     AS valores_ficha_retorno_elegivel,
    eleg.qtd_exame         AS qtd_exame_retorno_elegivel,
    eleg.secao             AS secao_retorno_elegivel,
    eleg.dias_entre_ficha  AS dias_entre_ficha_elegivel

FROM dados_laudos flr
INNER JOIN dados_birads brd
    ON flr.ficha = brd.ficha
    AND flr.id_exame = brd.id_exame
    AND flr.id_marca = brd.id_marca
    AND flr.sequencial = brd.sequencial    
LEFT JOIN refined.saude_preventiva.pardini_retorno_elegivel_ficha eleg
    ON eleg.ficha_origem = flr.ficha
    AND eleg.id_cliente = flr.id_cliente
    AND eleg.linha_cuidado = flr.linha_cuidado
{filtro_ativacao}
"""

df_spk = spark.sql(query.format(where_clause=where_clause, filtro_ativacao=""))

df_spk_ativacao = spark.sql(
    query.format(where_clause="", filtro_ativacao=filtro_ativacao)
)

## Função para calcular data prevista de retorno
Esta função adiciona a coluna `dth_previsao_retorno` ao DataFrame, baseada no valor de BI-RADS. Para BI-RADS 1 ou 2, adiciona 360 dias à data do pedido; para BI-RADS 3, adiciona 180 dias; para outros valores, retorna None. Utiliza funções do PySpark para manipulação de datas e lógica condicional.

In [ ]:
def calcular_data_prevista(df_spk: DataFrame):
    """
    Adiciona uma coluna 'dth_previsao_retorno' ao DataFrame com base na coluna 'BIRADS'.

    - Para BIRADS 1 ou 2, adiciona 360 dias à data da coluna 'dth_pedido'.
    - Para BIRADS 3, adiciona 180 dias à data da coluna 'dth_pedido'.
    - Para outros valores de BIRADS, define 'dth_previsao_retorno' como None.

    Parâmetros:
    df_spk (DataFrame): O DataFrame Spark contendo os dados de entrada.

    Retorna:
    DataFrame: O DataFrame atualizado com a nova coluna 'dth_previsao_retorno'.
    """
    df_spk = df_spk.withColumn(
        'dth_previsao_retorno',
        when(
            col('BIRADS').isin([1, 2]),
            expr("date_add(dth_pedido, 360)")
        ).when(
            col('BIRADS') == 3,
            expr("date_add(dth_pedido, 180)")  
        ).otherwise(None)
    )
    return df_spk

## Função para transformar campos do DataFrame
Esta função aplica diversas transformações ao DataFrame:
- Verifica se o DataFrame está vazio e registra aviso;
- Adiciona coluna `retorno_cliente` com valores baseados em BI-RADS (12 meses para 1/2, 6 meses para 3, 0 para outros);
- Calcula a data prevista de retorno usando a função anterior;
- Converte colunas de datas para timestamp;
- Calcula a diferença em dias entre a data prevista de retorno e a data do pedido.
Essas transformações facilitam a análise e o envio de notificações.

In [ ]:
def transform_fields(df_spk: DataFrame) -> DataFrame:
  """
  Transforma os campos do DataFrame fornecido.

  - Verifica se o DataFrame está vazio. Se estiver, registra um aviso e retorna o DataFrame sem alterações.
  - Adiciona uma coluna 'retorno_cliente' com valores baseados na coluna 'BIRADS':
    - 12 meses para BIRADS 1 ou 2
    - 6 meses para BIRADS 3
    - 0 para outros valores
  - Calcula a data prevista de retorno usando a função 'calcular_data_prevista'.
  - Converte a coluna 'dth_pedido_retorno_elegivel' para o tipo timestamp.
  - Converte a coluna 'dth_previsao_retorno' para o tipo timestamp.
  - Calcula a diferença em dias entre 'dth_previsao_retorno' e 'dth_pedido', armazenando o resultado na coluna 'dias_ate_retorno'.

  Parâmetros:
  df_spk (DataFrame): O DataFrame a ser transformado.

  Retorna:
  DataFrame: O DataFrame transformado com as novas colunas.
  """

  if df_spk.isEmpty():
      logger.warning("No Data Found!")
      return df_spk

  df_spk = df_spk.withColumn(
      'retorno_cliente',
      when(col('BIRADS').isin([1, 2]), 12).when(col('BIRADS') == 3, 6).otherwise(0)
  )

  df_spk = calcular_data_prevista(df_spk)
  df_spk = df_spk.withColumn('dth_pedido_retorno_elegivel', to_timestamp(col('dth_pedido_retorno_elegivel')))
  df_spk = df_spk.withColumn('dth_previsao_retorno', to_timestamp(col('dth_previsao_retorno')))
  df_spk = df_spk.withColumn('dias_ate_retorno', datediff(to_date(col('dth_previsao_retorno')), to_date(col('dth_pedido'))))
  return df_spk

## Função para tratamento de erros e alerta
Esta função trata exceções durante a persistência dos dados. Em caso de erro, formata o traceback, envia alerta para o Sentinel (monitoramento), imprime o erro no console e relança a exceção. Isso garante rastreabilidade e monitoramento automático de falhas.

In [ ]:
WEBHOOK_DS_AI_BUSINESS_STG = 'prd'

def error_message(e):
    """
    Envia alerta para o Sentinel e exibe o traceback em caso de erro ao salvar dados.

    Parâmetros:
        e (Exception): Exceção capturada.

    Comportamento:
        - Formata o traceback do erro.
        - Envia alerta para o Sentinel com detalhes do erro.
        - Exibe o traceback no console.
        - Relança a exceção.
    """
    error_message = traceback.format_exc()
    summary_message = f"""Erro ao salvar dados.\n{error_message}"""
    sentinela_ds_ai_business = Sentinel(
        project_name='Monitor_Linhas_Cuidado_Mama',
        env_type=WEBHOOK_DS_AI_BUSINESS_STG,
        task_title='Pardini Mama'
    )
    sentinela_ds_ai_business.alerta_sentinela(
        categoria='Alerta', 
        mensagem=summary_message,
        job_id_descritivo='1_pardini_mama_birads'
    )
    traceback.print_exc()
    raise e

## Funções para salvar dados nas tabelas Delta
Estas funções realizam a persistência dos dados processados:
- `insert_data`: Insere o DataFrame na tabela Delta, sobrescrevendo dados existentes e atualizando o schema;
- `merge_data`: Realiza merge (upsert) dos dados, atualizando registros existentes e inserindo novos, usando SQL;
- `save_data`: Decide entre insert ou merge conforme existência da tabela e verifica se o DataFrame está vazio.
Todas as funções utilizam tratamento de erros e logging.

In [ ]:
def insert_data(df_spk: DataFrame, table_name: str):
    """
    Insere os dados do DataFrame na tabela Delta especificada, sobrescrevendo o conteúdo existente.
    
    Parâmetros:
        df_spk (DataFrame): DataFrame a ser salvo.
        table_name (str): Nome da tabela Delta de destino.
    """
    try:
        logger.info(f"Inserting Data: {table_name}")
        (
            df_spk.write
                .mode('overwrite')
                .option('mergeSchema','true')
                .option('overwriteSchema','true')
                .format('delta')
                .partitionBy('_datestamp')
                .saveAsTable(table_name)
        )
    except Exception as e:
        error_message(e)
 
def merge_data(df_spk: DataFrame, table_name: str):
    """
    Realiza merge dos dados do DataFrame na tabela Delta, atualizando registros existentes e inserindo novos.
    
    Parâmetros:
        df_spk (DataFrame): DataFrame com os dados incrementais.
        table_name (str): Nome da tabela Delta de destino.
    """
    try:
        logger.info(f"Merging Data: {table_name}")
        df_spk.createOrReplaceTempView(f"increment_birads")
        merge_query = f"""
            MERGE INTO {table_name} AS target
            USING increment_birads AS source
                ON target.ficha = source.ficha
                AND target.sequencial = source.sequencial
                AND target.sigla_exame = source.sigla_exame
            WHEN MATCHED THEN
                UPDATE SET *
            WHEN NOT MATCHED THEN
                INSERT *
        """
        spark.sql(merge_query)
    except Exception as e:
        error_message(e)
 
def save_data(df_spk: DataFrame, table_name: str):
    """
    Salva os dados do DataFrame na tabela Delta, utilizando merge se a tabela existir ou insert se não existir.
    
    Parâmetros:
        df_spk (DataFrame): DataFrame a ser salvo.
        table_name (str): Nome da tabela Delta de destino.
    """
    if df_spk.isEmpty():
        return None
 
    if spark.catalog.tableExists(table_name):
        merge_data(df_spk, table_name)
    else:
        insert_data(df_spk, table_name)

## Aplicação das transformações nos DataFrames
Aplica as funções de transformação nos DataFrames extraídos das consultas SQL. Isso garante que todos os campos necessários estejam presentes e corretamente formatados para persistência e análise.

In [ ]:
df_spk = transform_fields(df_spk)
df_spk_ativacao = transform_fields(df_spk_ativacao)

## Remoção de duplicados na lista de ativação
Remove duplicados do DataFrame de ativação, considerando apenas a ficha para notificação. Isso garante que cada paciente receba apenas uma notificação por ficha, evitando notificações duplicadas.

In [ ]:
# Excluir duplicados para considerar apenas a ficha na ativação e não os exames (itens). Assim vamos enviar apenas 
# 1 push por ficha
df_spk_ativacao = df_spk_ativacao.dropDuplicates(['ficha'])

## Impressão da quantidade de laudos salvos
Imprime a quantidade de laudos salvos nas tabelas principais e de ativação, permitindo o acompanhamento do volume de dados processados e persistidos.

In [ ]:
print('quantidade de laudos salvos na tabela',df_spk.count())
print('quantidade de laudos salvos na tabela de ativação', df_spk_ativacao.count())

## Salvamento dos dados nas tabelas Delta
Salva os dados processados nas tabelas Delta, utilizando merge ou insert conforme necessário. O DataFrame completo é salvo na tabela principal e o DataFrame de ativação na tabela de ativação.

In [ ]:
save_data(df_spk, table_birads)
insert_data(df_spk_ativacao, table_birads_ativacao)